<a href="https://colab.research.google.com/github/absolutelydawn/TestNeoX/blob/main/KoGPT_kakaos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install torch # Pytorch 기반의
!pip install transformers # transformers 에 속해있는 KoGPT모델
!pip install pandas # pandas로 데이터 처리
!pip install nltk # BLEU 점수를 계산하기 위해 Natural Language Toolkit (NLTK)를 사용
!pip install accelerate


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 6.2 MB/s eta 0:00:00


In [3]:
!pip install transformers
!pip install accelerate

  Using cached transformers-4.33.1-py3-none-any.whl (7.6 MB)


In [ ]:
# 런타임 만료로 실행 안됨.. 로컬 환경(Pycharm)에서 실행되는 코드로 변경 후 실행해볼것
# finetunning KoGPT(카카오브레인)
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, AdamW
from torch.utils.data import Dataset, DataLoader
from torch.nn import CrossEntropyLoss
from nltk.translate.bleu_score import corpus_bleu
import urllib.request

# 데이터 파일 다운로드
data_url = 'https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv'
file_name = 'Chatbot_data.csv'
urllib.request.urlretrieve(data_url, file_name)

# 데이터를 불러와서 DataFrame으로 읽기
df_train = pd.read_csv(file_name)

# 데이터 확인
print(df_train.head())

# 데이터 로딩 (훈련 데이터와 별도의 테스트 데이터 사용)
df_test = pd.read_csv('Chatbot_test_data.csv')  # 테스트 데이터 파일 사용

text_data_train = df_train['Q']
labels_train = df_train['A']

text_data_test = df_test['Q']
labels_test = df_test['A']

# 모델 및 토크나이저 로딩
tokenizer = AutoTokenizer.from_pretrained(
    'kakaobrain/kogpt', revision='KoGPT6B-ryan1.5b-float16',
    bos_token='[BOS]', eos_token='[EOS]', unk_token='[UNK]', pad_token='[PAD]', mask_token='[MASK]'
)

model = AutoModelForCausalLM.from_pretrained(
    'kakaobrain/kogpt', revision='KoGPT6B-ryan1.5b-float16',
    pad_token_id=tokenizer.eos_token_id,
    torch_dtype='auto', low_cpu_mem_usage=True
).to(device='cuda' if torch.cuda.is_available() else 'cpu')
_ = model.eval()

# 하이퍼파라미터 설정
batch_size = 4
learning_rate = 0.0001
epochs = 5

# 데이터로더 설정 (테스트 데이터로더 추가)
train_dataset = ChatbotDataset(text_data_train, tokenizer)
test_dataset = ChatbotDataset(text_data_test, tokenizer)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# 옵티마이저와 손실 함수 설정
optimizer = AdamW(model.parameters(), lr=learning_rate)
loss_fn = CrossEntropyLoss()

# 학습 루프
for epoch in range(epochs):
    model.train()
    total_loss = 0.0
    for batch in train_dataloader:
        optimizer.zero_grad()
        inputs = batch.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
        outputs = model(inputs, labels=inputs)
        loss = loss_fn(outputs.logits.view(-1, outputs.logits.size(-1)), inputs.view(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    average_loss = total_loss / len(train_dataloader)
    print(f'Epoch [{epoch + 1}/{epochs}] - Loss: {average_loss:.4f}')

# 모델 성능 평가
model.eval()
references = []  # 정답 문장 리스트
hypotheses = []  # 생성된 문장 리스트

for batch in test_dataloader:
    inputs = batch.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
    generated_ids = model.generate(inputs, max_length=50, num_return_sequences=1)

    for gen_id in generated_ids:
        gen_text = tokenizer.decode(gen_id, skip_special_tokens=True)
        hypotheses.append(gen_text)

    for label in batch.tolist():
        ref_text = tokenizer.decode(label, skip_special_tokens=True)
        references.append([ref_text])

# 모델 저장
model_save_path = "chatbot_model.pth"
torch.save(model.state_dict(), model_save_path)

# BLEU 점수 계산
bleu_score = corpus_bleu(references, hypotheses)
print(f'BLEU Score: {bleu_score:.4f}')



                 Q            A  label
0           12시 땡!   하루가 또 가네요.      0
1      1지망 학교 떨어졌어    위로해 드립니다.      0
2     3박4일 놀러가고 싶다  여행은 언제나 좋죠.      0
3  3박4일 정도 놀러가고 싶다  여행은 언제나 좋죠.      0
4          PPL 심하네   눈살이 찌푸려지죠.      0


In [ ]:
# 모델 확인(간단하게)
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import sys

# 모델 및 토크나이저 로딩
tokenizer = AutoTokenizer.from_pretrained(
    'kakaobrain/kogpt', revision='KoGPT6B-ryan1.5b-float16',
    bos_token='[BOS]', eos_token='[EOS]', unk_token='[UNK]', pad_token='[PAD]', mask_token='[MASK]'
)

model = AutoModelForCausalLM.from_pretrained(
    'kakaobrain/kogpt', revision='KoGPT6B-ryan1.5b-float16',
    pad_token_id=tokenizer.eos_token_id,
    torch_dtype='auto', low_cpu_mem_usage=True
).to(device='cuda' if torch.cuda.is_available() else 'cpu')
_ = model.eval()

# 저장된 모델 로드
model_save_path = "chatbot_model.pth"
model.load_state_dict(torch.load(model_save_path))
model.eval()

# 사용자 입력 받기 및 대답 생성
while True:
    user_input = input("사용자: ")
    if user_input.lower() == 'exit':
        break

    # 사용자 입력을 토큰화하여 모델 입력으로 변환
    input_ids = tokenizer.encode(user_input, return_tensors="pt").to(device)

    # 모델을 사용하여 답변 생성
    generated_ids = model.generate(input_ids, max_length=50, num_return_sequences=1)

    # 생성된 답변 출력
    chatbot_response = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    print("Chatbot:", chatbot_response)
